In [ ]:
from numpy import *
from Feature_Extraction import *
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import spearmanr

IDX = ["0.5k", "1k", "2k", "4k", "8k", "13.5k"]

FILE = open("Hyperparameter_Candidates_BoostedRT.txt", "r")
Models = FILE.readlines()
FILE.close()

In [ ]:
# -----------------------------------------------------
# Nested Cross-validation
# -----------------------------------------------------
for idx in range(len(IDX)):

    NestedCV_Results = []
    # Outer 10-fold CV
    for f in range(10):
        
        Model_Selection_Results = [0.0] * len(Models)
        # Inner 5-fold CV
        for i in range(5):
            # Load inner CV training data
            FILE = open("DATA/%s_NestedCV_OuterFold%d_InnerFold%d_Train.txt" % (IDX[idx], f, i), "r")
            data = FILE.readlines()
            TRAIN_DATA_X, TRAIN_DATA_Y = Feature_Extraction(data)
            FILE.close()

            # Load inner CV validation data
            FILE = open("DATA/%s_NestedCV_OuterFold%d_InnerFold%d_Validation.txt" % (IDX[idx], f, i), "r")
            data = FILE.readlines()
            VAL_DATA_X, VAL_DATA_Y = Feature_Extraction(data)
            FILE.close()
                
            # Model hyperparameter selection
            for m in range(len(Models)):   
                Tokens = Models[m].strip().split()
                model = GradientBoostingRegressor(n_estimators = int(Tokens[0]), max_depth = int(Tokens[1]), min_samples_split = int(Tokens[2]), min_samples_leaf = int(Tokens[3]), max_features = Tokens[4])
                
                # Training
                model.fit(TRAIN_DATA_X, TRAIN_DATA_Y) 

                # Validation
                val_result = spearmanr(VAL_DATA_Y, model.predict(VAL_DATA_X))[0]
                Model_Selection_Results[m] += val_result
            
            
        # Select best performing model from inner 5-fold CV     
        Best_Model = argmax(Model_Selection_Results)
        Tokens = Models[BEST_Model].strip().split()
        model = GradientBoostingRegressor(n_estimators = int(Tokens[0]), max_depth = int(Tokens[1]), min_samples_split = int(Tokens[2]), min_samples_leaf = int(Tokens[3]), max_features = Tokens[4])

        # Load outer CV training data
        FILE = open("DATA/%s_NestedCV_OuterFold%d_Train.txt" % (IDX[idx], f), "r")
        data = FILE.readlines()
        TRAIN_DATA_X, TRAIN_DATA_Y = Feature_Extraction(data)
        FILE.close()

        # Load outer CV validation data
        FILE = open("DATA/NestedCV_OuterFold%d_Validation.txt" % (f), "r")
        data = FILE.readlines()
        VAL_DATA_X, VAL_DATA_Y = Feature_Extraction(data)
        FILE.close()
                
        # Training
        model.fit(TRAIN_DATA_X, TRAIN_DATA_Y) 

        # Validation
        val_result = spearmanr(VAL_DATA_Y, model.predict(VAL_DATA_X))[0]
        NestedCV_Results.append(val_result)
        
    print(IDX[idx], average(NestedCV_Results))